In [267]:
import pandas as pd
import numpy as np
import requests
import sys
print(sys.path)
from time import time

['', '/Users/Seth/anaconda/lib/python3.5/site-packages/nltk-3.2.1-py3.5.egg', '/Users/Seth/anaconda/lib/python35.zip', '/Users/Seth/anaconda/lib/python3.5', '/Users/Seth/anaconda/lib/python3.5/plat-darwin', '/Users/Seth/anaconda/lib/python3.5/lib-dynload', '/Users/Seth/.local/lib/python3.5/site-packages', '/Users/Seth/anaconda/lib/python3.5/site-packages/Sphinx-1.4.1-py3.5.egg', '/Users/Seth/anaconda/lib/python3.5/site-packages/setuptools-23.0.0-py3.5.egg', '/Users/Seth/anaconda/lib/python3.5/site-packages', '/Users/Seth/anaconda/lib/python3.5/site-packages/aeosa', '/Users/Seth/anaconda/lib/python3.5/site-packages/IPython/extensions', '/Users/Seth/.ipython']


In [246]:
DATA_FILE = "data/Mammalia-15040-df-fixedage.csv"
OUT_FILE = 'data/Mammalia-15040-df-images.csv'
print(OUT_FILE)
#DATA_FILE = "data/Chordata-2499-df-fixedage.csv"
#OUT_FILE = 'data/Chordata-2499-df-images.csv'

data/Mammalia-15040-df-images.csv


In [53]:
##################################
# https://gist.github.com/kekeblom/204a609ee295c81c3cc202ecbe68752c
##################################

import os
import re
import time
import argparse
import requests
import io
import hashlib
import itertools
import base64
from PIL import Image
from multiprocessing import Pool
from pyvirtualdisplay import Display
from selenium import webdriver

In [3]:
argument_parser = argparse.ArgumentParser(description='Download images using google image search')
argument_parser.add_argument('query', metavar='query', type=str, help='The query to download images from')
argument_parser.add_argument('--count', metavar='count', default=100, type=int, help='How many images to fetch')
argument_parser.add_argument('--label', metavar='label', type=str, help="The directory in which to store the images (images/<label>)", required=True)

_StoreAction(option_strings=['--label'], dest='label', nargs=None, const=None, default=None, type=<class 'str'>, choices=None, help='The directory in which to store the images (images/<label>)', metavar='label')

data/Mammalia-15040


In [61]:
def ensure_directory(path):
    if not os.path.exists(path):
        os.mkdir(path)

def largest_file(dir_path):
    def parse_num(filename):
        match = re.search('\d+', filename)
        if match:
            return int(match.group(0))

    files = os.listdir(dir_path)
    if len(files) != 0:
        return max(filter(lambda x: x, map(parse_num, files)))
    else:
        return 0

def fetch_image_urls(query, images_to_download):
    image_urls = set()
    
    # pyvirtualdisplay :: https://stackoverflow.com/questions/26070834/how-to-fix-selenium-webdriverexception-the-browser-appears-to-have-exited-befor
    display = Display(visible=0, size=(1024, 768))
    display.start()

    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"
    #browser = webdriver.Firefox()
    browser = webdriver.Firefox(executable_path=r'/Users/Seth/anaconda/lib/python3.5/geckodriver')
    browser.get(search_url.format(q=query))
    def scroll_to_bottom():
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

    image_count = len(image_urls)
    delta = 0
    while image_count < images_to_download:
        print("Found:", len(image_urls), "images")
        #scroll_to_bottom()

        #images = browser.find_elements_by_css_selector("img.rg_ic")
        images = browser.find_elements_by_css_selector("img")
        for img in images:
            image_urls.add(img.get_attribute('src'))
        delta = len(image_urls) - image_count
        image_count = len(image_urls)

        if delta == 0:
            print("Can't find more images")
            break

        fetch_more_button = browser.find_element_by_css_selector(".ksb._kvc")
        if fetch_more_button:
            browser.execute_script("document.querySelector('.ksb._kvc').click();")
            scroll_to_bottom()

    #browser.quit()
    browser.close()
    display.stop()
    return image_urls

def persist_image(dir_image_src):
    label_directory = dir_image_src[0]
    image_src = dir_image_src[1]

    size = (256, 256)
    try:
        image_content = requests.get(image_src).content
    except requests.exceptions.InvalidSchema:
        # image is probably base64 encoded
        image_data = re.sub('^data:image/.+;base64,', '', image_src)
        image_content = base64.b64decode(image_data)
    except Exception as e:
        print("could not read", e, image_src)
        return False

    image_file = io.BytesIO(image_content)
    image = Image.open(image_file).convert('RGB')
    resized = image.resize(size)
    with open(label_directory + hashlib.sha1(image_content).hexdigest() + ".jpg", 'wb')  as f:
        resized.save(f, "JPEG", quality=85)

    return True


### if __name__ == '__main__':
###     args = argument_parser.parse_args()
### 
###     ensure_directory('./images/')
### 
###     query_directory = './images/' + args.label + "/"
###     ensure_directory(query_directory)
### 
###     image_urls = fetch_image_urls(args.query, args.count)
### 
###     values = [item for item in zip(itertools.cycle([query_directory]), image_urls)]
### 
###     print("image count", len(image_urls))
### 
###     pool = Pool(12)
###     results = pool.map(persist_image, values)
###     print("Images downloaded: ", len([r for r in results if r]))

In [62]:
first_query = DATA_FILE.replace("data/", "").split("-")[0]
ROOT_PATH = './'
args_to_parse = "{} --count 1 --label {}".format(first_query, OUT_FILE)
args = argument_parser.parse_args(args_to_parse.split(' '))
print(args)
ensure_directory(ROOT_PATH)
query_directory = ROOT_PATH + args.label + "/"
ensure_directory(query_directory)

Namespace(count=1, label='data/Mammalia-15040/', query='Mammalia')


In [64]:
#image_urls = fetch_image_urls(args.query, args.count)
#image_urls

In [221]:
import base64
import requests
from resizeimage import resizeimage
from PIL import Image
import io

def bytes_to_img(img_bytes, resize=[]):
    image = Image.open(io.BytesIO(img_bytes))
    
    if len(resize) == 2:
        image = resizeimage.resize_contain(image, resize)
    elif len(resize) != 0:
        raise ValueError("resize argument takes a list of length 2 (height x width)")
    
    return image
    
def get_image_base64(img_bytes=None, url=None, resize=[]):
    if url:
        img_bytes = requests.get(url).content
    
    if len(resize) == 2:
        img = bytes_to_img(img_bytes, resize=resize)
        buffered = io.BytesIO()
        img.save(buffered, format="JPEG")
        img_bytes = buffered.getvalue()
    elif len(resize) != 0:
        raise ValueError("resize argument takes a list of length 2 (height x width)")
        
    return base64.b64encode(img_bytes)

def get_image_new(image_src):
    image_data = re.sub('^data:image/.+;base64,', '', image_src)
    image_content = base64.b64decode(image_data)
    return image_content


In [245]:
####### TOP OF PROGRAM?
display = Display(visible=0, size=(1024, 768))
display.start()
browser = webdriver.Firefox(executable_path=r'/Users/Seth/anaconda/lib/python3.5/geckodriver')

In [237]:
######### SCRAPE DATA #########

def get_img_from_query(query, skip=10, resize=[150,150], show=False):
    # pyvirtualdisplay :: https://stackoverflow.com/questions/26070834/how-to-fix-selenium-webdriverexception-the-browser-appears-to-have-exited-befor

    ### display = Display(visible=0, size=(1024, 768))
    ### display.start()
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"
    #browser = webdriver.Firefox()
    ### browser = webdriver.Firefox(executable_path=r'/Users/Seth/anaconda/lib/python3.5/geckodriver')

    browser.get(search_url.format(q=query))
    
    # get images
    images = browser.find_elements_by_css_selector("img")
    
    ### browser.close()
    ### display.stop()
    
    image_urls = list()
    
    for img in images:
        if img.get_attribute('src') is not None:
            image_urls.append(img.get_attribute('src'))
    
    if show:
        return bytes_to_img(get_image_new(image_urls[skip+1]), resize=resize)
    else:
        return get_image_base64(get_image_new(image_urls[skip+1]), resize=resize)


In [270]:
get_img_from_query("ugly", show=True)

<PIL.Image.Image image mode=RGB size=150x150 at 0x119A6A668>


In [242]:
get_img_from_query("ugly")

b'/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/2wBDAQkJCQwLDBgNDRgyIRwhMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjL/wAARCACWAJYDASIAAhEBAxEB/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/8QAHwEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAECAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRomJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4uPk5ebn6Onq8vP09fb3+Pn6/9oADAMBAAIRAxEAPwD3+iiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAbJIkMTyyuqRoCzMxwFA6knsKorrujt93VbE/S4T/GjXV3+HtSX1tZR/44a8ks7AAj5a4sXi3QaSV7kt2PXxq+mN01G0P0nX/GnDU7A9L62P

In [271]:
##### BOTTOM OF SCRIPT
browser.close()
display.stop()

<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1024x768x24', ':1025'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1024x768x24', ':1025'] oserror=None return_code=1 stdout="" stderr="_XSERVTransmkdir: ERROR: euid != 0,directory /tmp/.X11-unix will not be created.
_XSERVTransSocketUNIXCreateListener: mkdir(/tmp/.X11-unix) failed, errno = 2
_XSERVTransMakeAllCOTSServerListeners: failed to create listener for local
(EE) 
Fatal server error:
(EE) Cannot establish any listening sockets - Make sure an X server isn't already running(EE) " timeout_happened=False>

In [278]:
df = pd.read_csv(OUT_FILE.replace(".csv", "-base64.csv"))
df.head()

,ancestor,depth,extinct,id,name,num_kids,phylesis,x,y,Begin,End,img_url,img_base64
0,14973,1,0,15040,Mammalia,2,0,0.985482,-0.609871,225.00,0.0,https://en.wikipedia.org/wiki/File:Mammal_Dive...,b'/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgH...
1,15040,2,2,15989,Triconodonts,0,0,0.626042,-1.029422,190.00,70.0,https://en.wikipedia.org/wiki/File:Blank_page_...,b'/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgH...
2,15040,2,0,15991,Monotremata,2,0,1.577066,-1.157830,210.00,0.0,https://en.wikipedia.org/wiki/File:Prototheria...,b'/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgH...
3,15991,3,0,16250,Tachyglossidae,4,0,1.934838,-1.035496,23.03,0.0,https://en.wikipedia.org/wiki/File:Short-beake...,b'/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgH...
4,16250,4,0,16251,Tachyglossus aculeatus,0,0,2.682316,-1.617251,11.50,0.0,https://en.wikipedia.org/wiki/File:Wild_shortb...,b'/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgH...


In [279]:
fallback_bill = '/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/2wBDAQkJCQwLDBgNDRgyIRwhMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjL/wAARCABkAGQDASIAAhEBAxEB/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/8QAHwEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAECAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRomJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4uPk5ebn6Onq8vP09fb3+Pn6/9oADAMBAAIRAxEAPwD3+szUf+Phf9z+prTrN1D/AI+F/wB3+ppx3NKfxFKsPxP4mt/DVlHLLE800zFYolOMkDkk9gOK3a82+JEQvta021MuyOOJncgZxlsf0qpN20OlIwJ/iVr8twxjmto4z0jSPBH4nmsafxV4guLv7Q2rXCyINyhXKqPYDpVj7FYxZKwK+P75JJGaa1tbdTBDgjJwo6VFns2cjxcdbJl7RPivqWm6i8Osbr+0I4KhVlQ+x6Eex/OvVvDvifTPE9kbnTpi2w4kicbXjPuP69K8Gv8Aws1wPtNlMN78mN+n4H/GtH4b3V3oPji2triN4hdfuHVuhDdD784rWNWjUj7r1W6N4Rm2rrR7fme76pqMelabNfSo7xxAEqmMnJA7/WuWb4k2an93ptwx/wBqRR/jW54rjabwrqCIjO5jGFUZJO4dq80tvC+uXJBj0u4APQuoQfriuWvOpGVoH0OU4PBVqEqmJeqdtZW6LzOO8UStJeJICVEhdtoPq2f619K+EP8AkSdB/wCwdb/+i1r5s8VQvBcQxSDa8ZdGHXBBGa+k/CH/ACJOg/8AYOt//Ra1VL+GjzM7/wB8nbbT8kbVFFFWeQFZuof69f8Ad/qa0qzdQ/4+F/3f6mqjuaUviMXV9Xs9EsHvL2QJGvAHdj6CvHdR8XHxFr076fbXxYgfJHgAKBgAk9j1rrfi1Zzy6VZXaEmCCRlkXsN2MH9MVwngNN2pX/8ArAfJV8xrlwA3YVNSTVzrgryUe5t23hvxBctxYLboxJLzPuP5L/jWifAuoIu+TVoo2AzhoVCnH/As11Gn6o7rteC/hUYBN0BlvoRVXUt93cF4Le3kYMQGuE3BRj/HvXM52dmdaoaaHEXWiaxpimTyY7qJTgm3cqw7/db/ABNZKakZ5reW3YG5trhZPLcbXXBz0NelwwyyQSG5W3DKOsO7aTj0NcbqXh+21DUkDxyM4bIKcHb7HsalKDldrXuRKEo7HtUUiyosiMCrDIKnINSDqK8b0l9X0LxTp+kaZqcsiXMm+SKWPOIwRye2du4546V7IpB5Fd0JcyucdSHJJxZ8+eNLK4utduVgiL7bmbPIGPmr6D8JxtF4O0ONxhl0+BSPcRrWNJ4S0Oa5kuJbESSyuXctI3JJyeM11dpFHb2cEMSBI441RFHQADAFZxjKKs9jqzOvh67U6d+bS97W0SWn3E1FFFM8oKzNQP8ApKj/AGB/M1p1laif9MX/AHB/M1UdzSl8Rlatptpq+nS2N9GXt5cblDFTwcjBHuK4e28H2nhfWjf2FxI0NwPIMUuCVzz97uOK9AnUleKxNXgW5sXjZckEMPqKKkeaLsehQSUlJmc81vBcQq4byAxy/Xn147CoYtQH2wm1Utalj88ifIR7d+tQ3y3CwWd3ZyplBh4z8obtjdg459jReWmoXsMcZMVtGeCRKZG/AAAD8a4oWaPSnd2sjRnvonSNI4yjE42nnBrEYR22oQTSybI87WLNgcnpn9PxrUkjECW0JZmkBbLN1IA71y+pX9vHbG5umb7IJguduR7E+3c+lTBc07HPWaTO20yHTU1Ga8iCvdyKIzKeuwdAPQV00JyorjdBgEuLjIKdVIPWutt5M8V6rioqyPNq253ZloVqRcwp/uivOvFvi5bBH07T5AbthiSVTxCPQf7X8vrXbeHv+RZ0rP8Az5w/+gCud1E5cq6F4jBVKWHhXnopPRfK9zSooooOAKydSOLxP9wfzNa1Y+qnF4n+4P5mqhuaU/iIiRis3USqRFu4q4ZKzNTkXyJC7BVVSxZjgADua2ijpjKzuctdapDZzrHKyrDKWIz2PGf6Vd/4SLR1tgFvFdhzhBuP6V5ve+Jo7nxIrRgS2UQ2qCMb/U11NpHo94iR22oTW8MzAtBIvHPUA/4GuWrhpJucFdHfDEqXuJ7FmW9uNYvCLXcizDyo89VX+JvrWN4keG4Yadbj/RLYGMn+83Qn+n1BrrLiax0uxme0UAhdiuOcnHAH864qWL92FHXp6/55z+XvXVluGU26slotvX/gHHjajiuTqyj4a8ZX3h5RaOi3NopwI3OCv+639Oa6XUfiR9qthBp4azLriSSQ/P8ARSOB9ev0rzmbDSyuv3S7Y/M0/YHjA/WnUhe8bk4at7Kaqcqlbo9joo2Egyp359OcmvoTQo3i8PaZHIjI6WkSsrDBBCDINfLEc0sDBo3KsO6nFfUPheRpfCWjSOxZ2sYGYk5JJjWuSND2Tvfc7c1zV42nCLjazb/A1qKKKo8QK53XpCmoR4/55D+Zroq4/wAZ6ja6U32y8k2RJEPqxycADuTWlP4i4bjJb+C1tZLi6lSGGNdzyOcACvIfGHjaTXGktLEvFp3Qk8NN7n0X2/OsvxN4uuvEMwRswWMZykKnOT6t6n9BWDvV/ukn8DWxbl0EtgWvIiO74/MV1djP9nYo/wBzvnt9f/r1y6Dqe45H4c11kcaXdqkuQu5QN3XAIx/Mf5zivSwKvFpdDCpK0vUqQmS/ZruWZ4oQWYRBiAoAwvTuABVhJbuOPAcyoY922X7y5zgA9+Oxz04ovIPJsggJRAVR2IPCk85x75/X2q3Iq+VKVxjywy4xjGCf8/0NegoQehyqUlucoAVZlbk5Oc0HevEYGO5Pb8KkIy7f7xps+BCSpOSMHjHJ4r5+o/ffqz0or3PkReSEyxYsx6k19R+E/wDkTtD/AOwfB/6LWvmFAAuPwr6e8J/8idof/YPg/wDRa1lV2RnM2KKKKwICvD/jlfFta03TgSAtsZyPqxUfyNe4V8/fHM48cWfP/MOj/wDRstXT+IaPOPujPap48EEelVt2Tt9RSxttVW7gYrpBMtIQHGfXFb+jzkWbQk/cOBXMsdo6/jWrp8xjuwuTiQf5/pXbl8uWry91/wAEyrv3VLs/z0OqEYd7cn13fd6Y5/z+A45qS9TCzkdNvGPZSP6j1/EU1TxaH1cdPfI4/wA+v4WbwKbdgT/yzYn8Rj+Y9ufQ1697GLV2/wCuhxC84J69aiu5VURqSBlsn6D/APWKdvwB9KqzoZpAS2FCFSMdc189BRcve2O6pJqNo7hHctJK23HlLxn+8f8ACvqjwnz4N0P/ALB9v/6LWvlRIkhGEG0dK+q/CX/Im6H/ANg+3/8ARa1jXt0I1tqbFFFFcwgr58+Opx43s/8AsGp/6Nkooq4bgeaNxIuPU1In3PxNFFdKEwH+rx6Nir8DESQt3Df0oorfDfxo+qJq/wAKXodjKMtGOwBb8QA38xTb0uttdSea5IjwuTnaMv0oor3J7f12Zzx3OQByo+lRljvb2bFFFfPHa9xjDBr6q8Jf8iZoX/YPt/8A0WtFFYVtkJmxRRRWAj//2Q=='
x = df.loc[df.name.str.contains("atten")].squeeze()['img_base64']
fallback_bill == ast.literal_eval(x).decode('utf-8')

True

In [285]:
####### TOP OF PROGRAM?
display = Display(visible=0, size=(1024, 768))
display.start()
browser = webdriver.Firefox(executable_path=r'/Users/Seth/anaconda/lib/python3.5/geckodriver')

In [286]:
start = time()
first = True
img_list = []
for i, row in df.iterrows():
    if fallback_bill == ast.literal_eval(row['img_base64']).decode('utf-8'):
        try:
            if first:
                print("first one: "+row['name'])
                get_img_from_query(row['name'], show=True)
                first = False
            img_list.append(get_img_from_query(row['name']))

        except:
            #print("-- Bill!")
            img_list.append(None)
    else:
        img_list.append(None)
        
    if i % 50 == 0:
        print("{}::{}".format(i, int(time()-start)), end= ' ', flush=True)

0::0 first one: Zaglossus bartoni
50::40 100::119 150::212 200::317 250::473 300::611 350::717 400::766 450::845 500::895 550::937 600::989 650::1101 700::1189 750::1324 800::1397 850::1461 900::1519 950::1672 1000::1761 1050::1853 1100::1891 1150::1920 1200::1963 1250::2017 1300::2104 1350::2164 1400::2235 1450::2335 1500::2413 1550::2486 1600::2548 1650::2627 

In [287]:
##### BOTTOM OF SCRIPT
browser.close()
display.stop()

<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1024x768x24', ':1031'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1024x768x24', ':1031'] oserror=None return_code=1 stdout="" stderr="_XSERVTransmkdir: ERROR: euid != 0,directory /tmp/.X11-unix will not be created.
_XSERVTransSocketUNIXCreateListener: mkdir(/tmp/.X11-unix) failed, errno = 2
_XSERVTransMakeAllCOTSServerListeners: failed to create listener for local
(EE) 
Fatal server error:
(EE) Cannot establish any listening sockets - Make sure an X server isn't already running(EE) " timeout_happened=False>

In [288]:
df['img_base64_filly'] = img_list
df.tail()

,ancestor,depth,extinct,id,name,num_kids,phylesis,x,y,Begin,End,img_url,img_base64,img_base64_filly
1683,22665,6,2,60665,Mammuthus lamarmorae,0,0,1.954326,-0.574537,2.0,0.0,https://en.wikipedia.org/wiki/File:Wikipedia-l...,b'/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgH...,b'/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgH...
1684,22665,6,2,60663,Mammuthus meridionalis,0,0,1.725616,-0.535250,2.0,0.0,https://en.wikipedia.org/wiki/File:Museum_of_N...,b'/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgH...,None
1685,22665,6,2,60664,Mammuthus primigenius,0,0,2.760513,0.743133,2.0,0.0,https://en.wikipedia.org/wiki/File:Siegsdorfer...,b'/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgH...,None
1686,16451,5,0,22666,Elephas maximus,0,0,0.568061,0.147761,4.0,0.0,https://en.wikipedia.org/wiki/File:Elephas_max...,b'/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgH...,None
1687,16451,5,0,22667,Loxodonta africana,0,0,2.022854,0.642325,4.0,0.0,https://en.wikipedia.org/wiki/File:African_Ele...,b'/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgH...,None


In [289]:
df.to_csv(OUT_FILE.replace(".csv", "-base64-filly.csv"), index=False)

In [290]:
OUT_FILE.replace(".csv", "-base64-filly.csv")

'data/Mammalia-15040-df-images-base64-filly.csv'